# SpaceX Falcon 9 First Stage Landing Prediction: A Data Science Capstone Project

## Launch Sites Locations Analysis with Folium

In [1]:
import folium
import wget
import pandas as pd

In [2]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

#### Marking all launch sites on a map

In [4]:
spacex_df=pd.read_csv("https://github.com/JayshreeMishra/Dataset_Data_Science_Project/raw/main/spacex_launch_geo.csv")
spacex_df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [8]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df=spacex_df[["Launch Site", "Lat", "Long", "class"]]
launch_site_df=spacex_df.groupby(["Launch Site"], as_index=False).first()
launch_site_df=launch_site_df[["Launch Site", "Lat", "Long"]]
launch_site_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [9]:
# We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [11]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [14]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# folium.Circle and folium.Marker for each launch site on the site map In addition Launch site name as a popup label
for lat, lng, label in zip(launch_site_df['Lat'], launch_site_df['Long'], launch_site_df['Launch Site']):
    coordinate = [lat, lng]
    circle = folium.Circle(coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup(label))
    marker = folium.map.Marker(
        coordinate, 
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0), 
            html='%s' % label,)
        )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

Now, we can explore the map by zooming in/out of the marked areas and address questions such as:

### Are all launch sites in proximity to the Equator line?

The ideal location for an orbital launch site is close to the equator, providing an easterly water overflight path for safety. However, SpaceX has strategically allocated its launch missions among four facilities, each serving specific needs.

- Kennedy Space Center Launch Complex 39A (LC-39A): Used for NASA launches.
- Cape Canaveral Space Launch Complex 40 (SLC-40): Used for US Space Force national security launches.
- Vandenberg Space Force Base Space Launch Complex 4 E (SLC-4E): Used for polar launches.
- Brownsville South Texas Launch Site (Starbase): Used for commercial launches.

While not all launch sites are in proximity to the equator line, SLC-4E, designed for polar launches, is situated at a latitude of 34.632834°.

### Are all launch sites in very close proximity to the coast?

Not all launch sites are very close to the coast. For example, LC-39A is located on the east coast of Florida, while SLC-4E is situated on the west coast of the United States.
United States.
United States.


#### Marking the success/failed launches for each site on the map

In [15]:
spacex_df.tail()

,Launch Site,Lat,Long,class
51,CCAFS SLC-40,28.563197,-80.57682,0
52,CCAFS SLC-40,28.563197,-80.57682,0
53,CCAFS SLC-40,28.563197,-80.57682,0
54,CCAFS SLC-40,28.563197,-80.57682,1
55,CCAFS SLC-40,28.563197,-80.57682,0


In [16]:
# create markers for all launch records. 
# If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)
marker_cluster= MarkerCluster()

In [17]:
# Creating a new column in launch_sites dataframe called marker_color to store the marker colors based on the class value
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


In [18]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame creating a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
for index, record in spacex_df.iterrows():
    lat, lng, label, color = record['Lat'], record['Long'], record['Launch Site'], record['marker_color']
    coordinate=[lat, lng]
    marker= folium.Marker(
        coordinate,
        icon= folium.Icon(color="white", icon_color=color),
        popup=label
    )
    marker_cluster.add_child(marker)

site_map

#### Calculating the distances between a launch site to its proximities

In [19]:
# Adding Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [20]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [21]:
coastline_lat = 28.56398
coastline_lon = -80.56809
launch_site_lat = 28.56321
launch_site_lon = -80.57683
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

In [22]:
# added folium.Marker to show the distance
coast_coordinates = [coastline_lat, coastline_lon]
distance_marker = folium.Marker(
    coast_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='%s' % "{:10.2f} KM".format(distance_coastline),)
    )
distance_marker.add_to(site_map)
site_map

In [23]:
# a PolyLine between a launch site to the selected coastline point
launch_site_coordinates = [launch_site_lat, launch_site_lon]
lines=folium.PolyLine(locations=[coast_coordinates, launch_site_coordinates], weight=1)
site_map.add_child(lines)

In [24]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
city_lat = 28.61208
city_lon = -80.80764
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

city_coordinates = [city_lat, city_lon]
distance_marker = folium.Marker(
    city_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='%s' % "{:10.2f} KM".format(distance_city),
        )
    )
distance_marker.add_to(site_map)

launch_site_coordinates = [launch_site_lat, launch_site_lon]
lines=folium.PolyLine(locations=[city_coordinates, launch_site_coordinates], weight=1)
site_map.add_child(lines)
site_map

**After plotting distance lines to the proximities, we can answer the following questions easily:**

* **Do launch sites keep certain distance away from cities?**
    The analysis and distance calculations in Task 3 provide information on the distances between launch sites and certain proximities such as coastlines and cities. The distances are calculated and marked on the map, allowing for an assessment of how far the launch sites are from these features.

* **Are launch sites in close proximity to railways?**
    The code demonstrates how to calculate the distance between a launch site and a point of interest such as a railway. By using the MousePosition to identify coordinates and calculating the distance, one can analyze whether launch sites are in close proximity to railways.

* **Are launch sites in close proximity to highways?**
    Similar to railways, the code allows for the calculation of distances between launch sites and points of interest such as highways. The distances are visualized on the map, providing insights into the proximity of launch sites to highways.

In summary, the provided code and analysis allow for a visual exploration of SpaceX launch sites, their proximity to various features, and insights into geographical patterns related to launch locations.